In [1]:
from bs4 import BeautifulSoup
import urllib.request,urllib.parse,urllib.error
import pandas as pd
import requests
from urllib.error import HTTPError
from google.colab import files
import json

In [2]:
df =pd.read_csv("/content/Amazon Scraping - Sheet1.csv")
Asin=df["Asin"].tolist()
Country=df["country"].tolist()

In [3]:
Product_Title={"Product_Title":[]}
Product_Image_URL={"Product_Image_URL":[]}
Price_Of_The_Product={"Price_Of_The_Product":[]}
Product_Details={"Product_Details":[]}
Data=[]

In [9]:
headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
for ind in range(100):
  url="https://www.amazon.{}/dp/{}".format(Country[ind],Asin[ind])
    # fhand=urllib.request.urlopen(url)
    # soup=BeautifulSoup(fhand,'html.parser')
  r=requests.get(url,headers=headers)
  if(r.status_code==404):
    print(url + " not available")
  elif (r.status_code==503):
    break
  else:
    soup=BeautifulSoup(r.content, 'html.parser')
    if(soup.find("span",id="productTitle")):
      productTitle=soup.find("span",id="productTitle").text  #Product Title
      Product_Title["Product_Title"].append(productTitle)
    if(soup.find("div",id="main-image-container")):
      imageLink=soup.find("div",id="main-image-container").find("img")["src"]   # Product Image link
      Product_Image_URL["Product_Image_URL"].append(imageLink)

    for i in soup.find_all("span",class_="a-button-inner"):
      if(i.find("span",class_="a-size-base a-color-price a-color-price")):
        price=i.find("span",class_="a-size-base a-color-price a-color-price").text
        Price_Of_The_Product["Price_Of_The_Product"].append(price.replace("\xa0",""))
      elif(i.find("span",class_="a-color-base")):
        price=i.find("span",class_="a-color-base").text
        Price_Of_The_Product["Price_Of_The_Product"].append(price.replace("\xa0",""))
    for i in soup.find_all("div",id="corePriceDisplay_desktop_feature_div"):
      if(i.find("span",class_="a-offscreen")):
        price=i.find("span",class_="a-offscreen").text
        Price_Of_The_Product["Price_Of_The_Product"].append(price.replace("\xa0",""))
    key=[]
    try:
      productDetails=soup.find("div",id="prodDetails").find("table",id="productDetails_techSpec_section_1")
      rows=productDetails.find_all('tr')
      for row in rows:
        key.append(row.find('th').text)
        key.append(row.find('td').text.strip().strip("\u200e"))
      Product_Details["Product_Details"].append(key)
    except:
      try:
        productDetails=soup.find("div",id="detailBullets_feature_div").find_all("span",class_="a-list-item")
        for i in productDetails:
          for j in i.text.split(":"):
            key.append(j.strip(" \n\u200e\u200f"))
        Product_Details["Product_Details"].append(key)
      except:
        pass


In [5]:

Data.append(Product_Title)
Data.append(Product_Image_URL)
Data.append(Price_Of_The_Product)
Data.append(Product_Details)


In [6]:
print(Data)

[{'Product_Title': ['        Old Spice Rasur Creme - 70 G (original) - Packung Von 2       ', ' Short Story: Violoncello und Klavier. (Cello-Bibliothek) ', ' Short story (dushkin/hegyi) violon ', ' Concerto grosso g-Moll: Weihnachtskonzert. op. 6/8. 2 Violinen, Violoncello, Streichorchester und Cembalo. Partitur. (Concertino) ', ' Concerto grosso 8 g opus 6 partitu ', ' Concertino B-Dur: Violine und Orchester. Klavierauszug mit Solostimme. (Edition Schott) ', ' Concertino bes violon ', ' Concerto in E flat "Dumbarton Oaks": für Kammerorchester. Kammerorchester. Studienpartitur. (Musik unserer Zeit) ', " Concert es kammerorchester parti musique d'ensemble ", ' Musik für Kinder: Im Fünftonraum. Vol. 1. Singstimme, Blockflöte und Schlagzeug. Sing- und Spielpartitur. (Orff-Schulwerk) ', ' Schulwerk vol. 1 (ted.) (orff/keetman) ', ' 5 antike Oden: nach Gedichten von Sappho. op. 57. mittlere Frauenstimme, Viola und Klavier. mittel. Stimmensatz. (Edition Schott) ', ' Funf antike oden op. 57 -

In [7]:
with open("mydata.json", "w") as final:
    json.dump(Data, final)
files.download('mydata.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>